In [10]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [16]:
## implementar a função que identifica a bandeira
## a função deve receber uma imagem e retornar a bandeira identificada
## a função deve retornar uma lista de tuplas, onde cada tupla contém as coordenadas de um dos cantos da bandeira
## [(PAIS, (x1, y2), (x2, y2)`),(PAIS, (x1, y2), (x2, y2)`),(PAIS, (x1, y2), (x2, y2)`),...]

def identifica_bandeira(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    bandeiras_encontradas = []
    vermelho_baixo1 = np.array([0, 120, 100])
    vermelho_alto1 = np.array([10, 255, 255])
    vermelho_baixo2 = np.array([170, 120, 100])
    vermelho_alto2 = np.array([180, 255, 255])
    branco_baixo = np.array([0, 0, 200])
    branco_alto = np.array([180, 30, 255])
    verde_baixo = np.array([35, 100, 100])
    verde_alto = np.array([85, 255, 255])
    laranja_baixo = np.array([10, 100, 100])
    laranja_alto = np.array([25, 255, 255])
    

    mask_vermelho = cv2.inRange(hsv, vermelho_baixo1, vermelho_alto1) + cv2.inRange(hsv, vermelho_baixo2, vermelho_alto2)
    mask_branco = cv2.inRange(hsv, branco_baixo, branco_alto)
    mask_verde = cv2.inRange(hsv, verde_baixo, verde_alto)
    mask_laranja = cv2.inRange(hsv, laranja_baixo, laranja_alto)
    
    mask_total = mask_vermelho | mask_branco | mask_verde | mask_laranja
    
    contornos, _ = cv2.findContours(mask_total, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contorno in contornos:
        if cv2.contourArea(contorno) < 500:
            continue
        
        x, y, w, h = cv2.boundingRect(contorno)
        x1, y1 = x, y  
        x2, y2 = x + w, y + h  
        
        regiao_hsv = hsv[y:y+h, x:x+w]
       
        total_pixels = w * h
        vermelho_pixels = cv2.countNonZero(cv2.inRange(regiao_hsv, vermelho_baixo1, vermelho_alto1) + cv2.inRange(regiao_hsv, vermelho_baixo2, vermelho_alto2))
        branco_pixels = cv2.countNonZero(cv2.inRange(regiao_hsv, branco_baixo, branco_alto))
        verde_pixels = cv2.countNonZero(cv2.inRange(regiao_hsv, verde_baixo, verde_alto))
        laranja_pixels = cv2.countNonZero(cv2.inRange(regiao_hsv, laranja_baixo, laranja_alto))
        
        proporcao_vermelho = vermelho_pixels / total_pixels
        proporcao_branco = branco_pixels / total_pixels
        proporcao_verde = verde_pixels / total_pixels
        proporcao_laranja = laranja_pixels / total_pixels
        
        if proporcao_vermelho > 0.4 and proporcao_branco > 0.4 and proporcao_verde < 0.1 and proporcao_laranja < 0.1:
            pais = "monaco"
        elif proporcao_vermelho > 0.5 and proporcao_branco > 0.2 and proporcao_verde < 0.1 and proporcao_laranja < 0.1:
            pais = "peru"
        elif proporcao_vermelho > 0.3 and proporcao_branco > 0.3 and proporcao_verde < 0.1 and proporcao_laranja < 0.1 and w > h:
            pais = "singapura"
        elif proporcao_verde > 0.2 and proporcao_branco > 0.2 and proporcao_laranja > 0.2 and proporcao_vermelho < 0.1:
            pais = "irlanda"
        elif proporcao_verde > 0.2 and proporcao_branco > 0.2 and proporcao_vermelho > 0.2 and proporcao_laranja < 0.1:
            pais = "italia"
        else:
            pais = "desconhecido"
        
        bandeiras_encontradas.append((pais, (x1, y1), (x2, y2)))
    
    return bandeiras_encontradas




In [33]:
## nao alterar essa função
# Função para desenhar as bandeiras identificadas
def draw_bandeiras(lista_bandeiras, bgr):
    try:
        for bandeira in lista_bandeiras:
            cv2.rectangle(bgr, bandeira[1], bandeira[2], (255, 0, 0), 5)
            cv2.putText(bgr, bandeira[0], bandeira[1], cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        return bgr
    except Exception as e:
        return bgr

In [ ]:
## Para testar a função identifica_bandeira

# Você pode testar a função identifica_bandeira com as imagens de teste fornecidas
# 
img = cv2.imread('img/teste1.png')
# img = cv2.imread('img/teste2.png')
# img = cv2.imread('img/teste3.png')
# img = cv2.imread('img/teste4.png')

# Chamar a função identifica_bandeira
resultado = identifica_bandeira(img)

# Imprimir o resultado da função identifica_bandeira 
print(resultado)

# Desenhar as bandeiras na imagem
bgr = draw_bandeiras(resultado, img)
plt.imshow(cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB))
plt.show()


